# Table of Contents

- [Imports](#Imports-and-Configuration)
- [Photo Downloads](#Photo-Download)

# Imports and Configuration

Import necessary packages and set API keys/ configure APIs

In [1]:
import pandas as pd
import numpy as np
import folium
import geocoder
import xml
import urllib
from PIL import Image
import matplotlib.pyplot as plt
import flickr_api as flickr
import googlemaps
import time
import requests
from googlemaps import convert
flickr.set_keys(api_key = '4ea697590240af7b4ced24c4e46b3d41', api_secret = '855a29a3e8cd2677')

In [2]:
# source https://github.com/alexis-mignon/python-flickr-api/wiki/Flickr-API-Keys-and-Authentication
a = flickr.auth.AuthHandler() # creates a new AuthHandler object
perms = "write" # set the required permissions
url = a.get_authorization_url(perms)
print(url)

https://www.flickr.com/services/oauth/authorize?oauth_token=72157707122717761-086fc3e8e7beba6d&perms=write


In [3]:
a.set_verifier('327dd0dcdbb54401') # copy your oauth_verifier tag here!
flickr.set_auth_handler(a)
a.save('.auth.txt')
flickr.set_auth_handler(".auth.txt")

# Photo Download

We're using the Flickr API here to get images to train the model on. We want to get images that belong to either 'food', 'view', 'park' or 'art' classes so we are using the tag parameter to filter returned images. 
For our neural network training we need 1000 images per class for training and 300 images per class for validation

In [5]:
# Looking for pictures with  the following tags
tags = ['food', 'coffee', 
        'art', 'view', 
        'park']

# Download photos 
# for each tag we need to download 1300 images (1000 to train and 300 to validate)
for tag_current in tags:
    print('downloading pictures for', tag_current)
    # dictionary with usernames and the number of photos we have from this user. 
    # this is done to ensure not getting spams/ too many pictures from one user
    # and also to get a variety of images
    user_photo_counts = {}
    
    # we will loop through pages
    page_number = 1 
    
    # initialize the urls list, to keep track of the number of images downloaded
    # food and coffee are in the same class, so we want half the number of images
    # also since we are moving images to train and validation directories based on filenames, 
    # we want the directory to contain equal number of 
    # food and coffee images in 0 - 1000 this is just to keep the classes balanced and not give our training set 
    # mostly food pictures and our validation set only pictures of coffee
    urls = []
    if ((tag_current == 'food') or (tag_current == 'coffee')):
        len_urls = 800
        if (tag_current == 'food'):
            file_number = 1
        else:
            file_number = 2
    else:
        len_urls = 1300
        file_number = 1
    while (len(urls) < len_urls):
        photo_search = flickr.Photo.search(api_key = '4ea697590240af7b4ced24c4e46b3d41',  
                                           per_page = 500,
                                           lat = 40.71455,
                                           lon = -74.00713,
                                           tags = tag_current,
                                           page = page_number,
                                           accuracy = 11,
                                           content_type = 1,
                                           media = 'photos') 
        for photo in photo_search:
            try:
                # update user photo count dictionary
                user_name = photo.getInfo()['owner'].username
                if user_name in user_photo_counts.keys():
                    user_photo_counts[user_name] += 1
                else:
                    user_photo_counts[user_name] = 1
                # keep photo only if we have less than 10 images from that person
                if user_photo_counts[user_name] <= 10:
                    url = photo.get('url_c')
                    urls.append(url)
                    # specify folder depending on the tag
                    if ((tag_current == 'food') or (tag_current == 'coffee')):
                        place = 'food'
                    elif (tag_current == 'art'):
                        place = 'culture'
                    elif tag_current == 'view':
                        place = 'view'
                    elif tag_current == 'park':
                        place = 'park'
                    elif tag_current == 'hotel':
                        place = 'hotel'
                    # get the path the picture will be saved to
                    path = '../picture_data/' + place + '/ ' + place + str(file_number) + '.jpg'
                    try:
                        urllib.request.urlretrieve(url, path)
                        image = Image.open(path)
                        image = image.resize((256, 256), Image.ANTIALIAS)
                        image.save(path)
                        if ((tag_current == 'food') or (tag_current == 'coffee')):
                            file_number += 2
                        else:
                            file_number += 1
                    except:
                        pass
            except:
                pass

        page_number += 1

downloading pictures for art
